In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset


In [2]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras


from datasets import load_dataset, load_metric

#raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
     

In [3]:
train_data = pd.read_excel('train.xlsx')
test_data = pd.read_excel('evaluation.xlsx')

In [4]:
test_data_ = pd.read_excel('evaluation.xlsx')

In [5]:
train_data.head()

,text,reason,label
0,this is an amazing app for online classes!but,good app for conducting online classes,1.0
1,very practical and easy to use,app is user-friendly,1.0
2,this app is very good for video conferencing.,good for video conferencing,1.0
3,i can not download this zoom app,unable to download zoom app,1.0
4,i am not able to download this app,want to download the app,1.0


In [6]:
train_data = train_data.drop(['label'], axis = 1)
test_data = test_data.drop(['label'], axis = 1)

In [7]:
train_data= Dataset.from_pandas(train_data)
test_data= Dataset.from_pandas(test_data)

In [8]:
train_data

Dataset({
    features: ['text', 'reason'],
    num_rows: 2061
})

In [9]:
raw_datasets =train_data.train_test_split(
    train_size=0.9, test_size=0.1
)

In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'reason'],
        num_rows: 1854
    })
    test: Dataset({
        features: ['text', 'reason'],
        num_rows: 207
    })
})

In [11]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 66  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH =16 # Maximum length of the output by the model
BATCH_SIZE = 4  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 3  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-base"


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

/home/rohit/anaconda3/envs/tf_new/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [13]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [14]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=66, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["reason"], max_length=16, truncation=True,padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)


  0%|          | 0/2 [00:00<?, ?ba/s]

/home/rohit/anaconda3/envs/tf_new/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

2023-01-14 00:07:28.287647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 00:07:28.292858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 00:07:28.293215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 00:07:28.294355: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [18]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [42]:
test_dataset

<PrefetchDataset shapes: {input_ids: (None, 66), attention_mask: (None, 66), labels: (None, 16), decoder_input_ids: (None, 16)}, types: {input_ids: tf.int64, attention_mask: tf.int64, labels: tf.int64, decoder_input_ids: tf.int64}>

In [19]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [20]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     for label in labels:
#         label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [21]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# For now we will use our test set as our validation_data
model.fit(
    train_dataset, validation_data=test_dataset,batch_size = 1, epochs=10, callbacks=callbacks
)


2023-01-14 00:07:43.696892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
464/464 [==============================] - 72s 118ms/step - loss: 2.0210 - val_loss: 0.6519


/home/rohit/anaconda3/envs/tf_new/lib/python3.8/site-packages/transformers/generation/tf_utils.py:1800: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-01-14 00:08:55.905707: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7f79ac007090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-14 00:08:55.905729: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): GeForce GTX 1080 Ti, Compute Capability 6.1
2023-01-14 00:08:55.940949: I tensorflow/compiler/jit/xla_compilation_cache.cc:363] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 2/10
464/464 [==============================] - 51s 111ms/step - loss: 0.6891 - val_loss: 0.5346
Epoch 3/10
464/464 [==============================] - 51s 111ms/step - loss: 0.5712 - val_loss: 0.4927
Epoch 4/10
464/464 [==============================] - 52s 111ms/step - loss: 0.5082 - val_loss: 0.4752
Epoch 5/10
464/464 [==============================] - 52s 111ms/step - loss: 0.4649 - val_loss: 0.4568
Epoch 6/10
464/464 [==============================] - 52s 111ms/step - loss: 0.4168 - val_loss: 0.4537
Epoch 7/10
464/464 [==============================] - 52s 111ms/step - loss: 0.3829 - val_loss: 0.4605
Epoch 8/10
464/464 [==============================] - 52s 111ms/step - loss: 0.3496 - val_loss: 0.4650
Epoch 9/10
464/464 [==============================] - 52s 111ms/step - loss: 0.3206 - val_loss: 0.4672
Epoch 10/10
464/464 [==============================] - 52s 111ms/step - loss: 0.2925 - val_loss: 0.4789


In [22]:
# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=66, return_tensors="pt")
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [23]:
#test dataset tokenize
test_data_new = test_data.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

/home/rohit/anaconda3/envs/tf_new/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [25]:
results = test_data.map(generate_summary, batched=True, batch_size=16, remove_columns=['text']) 

  0%|          | 0/563 [00:00<?, ?ba/s]

In [26]:
results['pred']

['video and crashing when i play video',
 'unable to connect to tv',
 'remote. helpful when and home working remotely. working remotely. working remotely.',
 'so called and missed call and mobile number and missed call and missed call and mobile number zoom',
 '. It is also one of my favorite apps.',
 'shows and movies. watching shows and movies. watching shows and movies. movies',
 'not having internet for a month and a half have not had internet for ',
 'video will be playing offline without adds',
 'ipad pro m1 is not good for video conferencing',
 'i can continue watching where i left off.',
 'issue takes more than one month to resolve.soomsoomsoomsoom',
 'unable to resolve issues',
 'play movies. I have to restart the app to play movies.',
 'the dramas...... has so much to watch..',
 'to record podcast episodes remotely!',
 'zoom is useful for home school or meetings at home. zoom is useful for home school or',
 'application, but the network is bothering., but that is not the case

In [27]:
results['reason']

['app crashes during playback',
 'want compatibility with more smart televisions',
 'good app for work',
 'receiving incorrect phone number message',
 'good for spending time',
 'good to watch shows',
 'unable to access live tv',
 'good app to download videos',
 'want to record on ipad',
 'unable to sync watched episodes',
 'zoom server crashes',
 'want to contact technical support',
 'unable to view all movies',
 'good to watch shows',
 'want to record audio when not the host',
 'good app for students',
 'facing network issues with app',
 'getting ads frequently',
 'good app for conducting online meeting',
 'unable to unsubscribe',
 'good for family and friends',
 'good app for conducting online meeting',
 'app is good to watch disney content',
 'good for watching movies and serials',
 'app is not good',
 'reinstalling did not work',
 'want to enter breakout room',
 'app is not working after latest update',
 'unable to use streaming service',
 'app subscription is not worthy',
 'good 

In [28]:
from rouge_score import rouge_scorer

rougeL = []
scorer = rouge_scorer.RougeScorer(['rougeL'])
for i in range(len(results['reason'])):
    rougeL.append(scorer.score(results['reason'][i],results['pred'][i])['rougeL'].fmeasure)

In [40]:
predict_res = []
for i in range(len(rougeL)):
    if rougeL[i] > 0.19:
        predict_res.append(1)
    else:
        predict_res.append(0)

In [41]:
from sklearn.metrics import classification_report,roc_auc_score
y_true = test_data_['label'].values

print(classification_report(y_true,predict_res))

              precision    recall  f1-score   support

         0.0       0.77      0.71      0.73      5999
         1.0       0.49      0.57      0.53      3001

    accuracy                           0.66      9000
   macro avg       0.63      0.64      0.63      9000
weighted avg       0.67      0.66      0.67      9000



In [59]:
#for 0.13 t5-small
roc_auc_score(y_true,predict_res)

0.6161282350790555

In [42]:
#for 0.19 t5-large
roc_auc_score(y_true,predict_res)

0.6365477218545642